In [1]:
import json
from openai import OpenAI
from groq import Groq
import os
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
import pickle
from requests.exceptions import HTTPError
import time
# from groq.exceptions import RateLimitError
from concurrent.futures import ThreadPoolExecutor

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client =  Groq(api_key = os.environ['GROQ_API_KEY'])

In [3]:
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

/home/codespace/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
with open('../data/vietnamese_rag/documents-with-ids1.json', 'rt') as f_in:
    documents1 = json.load(f_in)

In [30]:
len(documents1)

1217

In [14]:
documents1[0].keys()

dict_keys(['group', 'context', 'question', 'answer', 'id'])

In [5]:
prompt_template = """
You emulate my assistant who works with me in a Q and A project .
Formulate 5 questions people might ask based on the record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. Make sure the questions should be in Vietnamese

The record:

question: {question}
answer: {answer}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [6]:
def generate_questions(doc):
    # Create a new dictionary excluding the first key-value pair
    doc_items = list(doc.items())[1:]  # Skip the first item
    doc_filtered = dict(doc_items)

    prompt = prompt_template.format(**doc_filtered)

    retries = 5
    for i in range(retries):
        try:
            response = client.chat.completions.create(
                model='Gemma2-9b-It',
                messages=[{"role": "user", "content": prompt}]
            )
            json_response = response.choices[0].message.content
            return json_response
        except HTTPError as e:
            if e.response.status_code == 429:  # Rate limit error
                retry_after = float(e.response.json()['error']['message'].split('in ')[-1].split('s')[0])
                time.sleep(retry_after)
            else:
                raise
        except Exception as e:
            if i < retries - 1:
                time.sleep(2 ** i)  # Exponential backoff
            else:
                raise

In [7]:
def map_progress(pool, seq, f):
    results = []

    with tqdm(total=len(seq)) as progress:
        futures = []

        for el in seq:
            future = pool.submit(f, el)
            future.add_done_callback(lambda p: progress.update())
            futures.append(future)

        for future in futures:
            result = future.result()
            results.append(result)

    return results

In [8]:

# Initialize ThreadPoolExecutor
pool = ThreadPoolExecutor(max_workers=6)

# Process documents in parallel
results = {}

In [9]:
def process_document(doc):
    doc_id = doc['id']
    if doc_id in results:
        return None

    questions = generate_questions(doc)
    return (doc_id, questions)

In [20]:
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[:30], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions


  2%|█▏                                               | 28/1127 [00:20<35:37,  1.94s/it]████████████████████████████████████████████████████████████████▍          | 28/30 [00:02<00:00, 10.40it/s]

In [21]:

# Print or save the results as needed
print(results)

{'75fafd29': '[\n"Minh Tú đã gặp khó khăn gì trong thử thách đi catwalk tại Asia\'s Next Top Model mùa 5?",\n"Điểm đến nào là thử thách khó khăn nhất đối với Minh Tú trong buổi thử thách đi catwalk?",\n"Vị trí của Minh Tú trong đêm chung kết Asia\'s Next Top Model mùa 5 là gì?",\n"Minh Tú đã thể hiện kỹ năng gì khi thực hiện thử thách đi catwalk trên tòa nhà cao tầng?",\n"Những thử thách nào đã giúp Minh Tú gặt hái thành công trong Asia\'s Next Top Model mùa 5?"\n] \n', '7adda7d7': '["Tại sao TP HCM có sương mù dày đặc vào sáng hôm nay?",\n"Những yếu tố nào dẫn đến sương mù dày đặc ở TP HCM?",\n"Tại sao sương mù lại xuất hiện vào ban đêm?",\n"Có sự liên quan giữa mưa lớn và hiện tượng sương mù\xa0không?",\n"Mức độ ô nhiễm không khí có ảnh hưởng đến sương mù\xa0không?"] \n\n\n', 'de6e7aa5': '{"question1":"Vụ nổ tại nhà máy hóa dầu xảy ra ở tỉnh nào ở Trung Quốc?" , "question2":"Ảnh hưởng ban đầu của vụ nổ là bao nhiêu người bị thương?" , "question3":"Vụ nổ xảy ra vào thời gian nào theo 

In [22]:
len(processed_results)

30

In [23]:

# from collections import defaultdict

# hashes = defaultdict(list)

# for doc in documents1:
#     doc_id = doc['id']
#     hashes[doc_id].append(doc)
# # hashes['75fafd29']

In [24]:
# len = 0
# for hash in hashes:
#     print(hash)
#     len += 1
#     if len == 30:
#         break
    

In [25]:
# for result in results:
#     print(result)

In [26]:
with open('../data/vietnamese_rag/ground_truth_data/ground_truth1.pkl', 'wb') as file:
    pickle.dump(results, file)

In [27]:
with open('../data/vietnamese_rag/ground_truth_data/ground_truth1.pkl', 'rb') as file:
    test = pickle.load(file)

In [29]:
type(test)

dict

In [31]:
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30:30*2], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions


100%|█████████████████████████| 30/30 [00:03<00:00,  9.56it/s]


In [35]:
def get_last_n_items(d, n):
    # Convert dictionary items to a list
    items = list(d.items())
    # Slice the list to get the last n items
    last_n_items = items[-n:]
    # Convert the sliced list back to a dictionary
    return dict(last_n_items)

In [36]:
last_30_items = get_last_n_items(results, 30)
len(last_30_items)

30

In [37]:
with open('../data/vietnamese_rag/ground_truth_data/ground_truth2.pkl', 'wb') as file:
    pickle.dump(last_30_items, file)

In [43]:
with open('../data/vietnamese_rag/ground_truth_data/ground_truth2.pkl', 'rb') as file:
    test = pickle.load(file)

In [44]:
test

{'1549e29b': '{"questions":["Lexus NX mới có lưới tản nhiệt dạng nào?","Lexus NX mới được trang bị màn hình thông tin và giải trí có kích thước bao nhiêu inch?","NX 300h và NX 300 F Sport có điểm khác biệt nào về phần đầu xe?","Có những thay đổi ở phần đuôi xe Lexus NX mới?","Lexus NX mới sử dụng động cơ loại nào?"]}  \n',
 'f55ad056': '["Huỳnh Thị Thuỳ Dung đại diện nước nào tham dự cuộc thi nào?", "Huỳnh Thị Thuỳ Dung đạt giải gì ở cuộc thi Hoa hậu Việt Nam 2016?", "Ngoài vị trí Á hậu 2, Huỳnh Thị Thuỳ Dung còn nhận giải gì?", "Năm Huỳnh Thị Thuỳ Dung đại diện Việt Nam tham dự Miss International?", "Huỳnh Thị Thuỳ Dung là người đẹp tài năng trong cuộc thi Hoa hậu Việt Nam nào?"] \n\n\n',
 '326bd715': '["Sở dĩ Tuấn Anh không thể tham gia buổi tập đội tuyển U22 Việt Nam là do lý do gì?", "Trường hợp Đức Huy không góp mặt trong buổi tập là do chấn thương gì?", "Chuyên gia vật lý trị liệu nào đang điều trị cho Tuấn Anh?", "Cầu thủ Tuấn Anh bị chấn thương gì?", "Được biết Đức Huy bị chấn 

In [45]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*2:30*3], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:02<00:00, 10.34it/s]


In [50]:
len(results)

30

In [51]:
with open('../data/vietnamese_rag/ground_truth_data/ground_truth3.pkl', 'wb') as file:
    pickle.dump(results, file)

In [10]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*3:30*4], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions

 40%|████████████████████▍                              | 12/30 [00:12<00:27,  1.53s/it]

In [11]:
with open('../data/vietnamese_rag/ground_truth_data/ground_truth4.pkl', 'wb') as file:
    pickle.dump(results, file)

In [10]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*4:30*5], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth5.pkl', 'wb') as file:
    pickle.dump(results, file)

100%|█████████████████████████| 30/30 [00:13<00:00,  2.42it/s]

In [11]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*5:30*6], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth6.pkl', 'wb') as file:
    pickle.dump(results, file)

100%|█████████████████████████| 30/30 [00:13<00:00,  2.16it/s]


In [ ]:
results = {}
# Use map_progress to process documents
processed_results = map_progress(pool, documents1[30*6:30*7], process_document)

# Store the results
for result in processed_results:
    if result is not None:
        doc_id, questions = result
        results[doc_id] = questions
with open('../data/vietnamese_rag/ground_truth_data/ground_truth7.pkl', 'wb') as file:
    pickle.dump(results, file)